## NGHIÊN CỨU ĐIỂN HÌNH-ĐÁNH GIÁ NGÂN HÀNG KHAI THÁC VĂN BẢN / PHÂN TÍCH KHIẾU NẠI

In [156]:

import pandas as pd  # Để làm việc với dữ liệu dạng bảng và chuỗi
import numpy as np   # Để xử lý dữ liệu số học và mảng
import seaborn as sns  # Để vẽ đồ thị và trực quan hóa dữ liệu
import matplotlib.pyplot as plt  # Để vẽ đồ thị

import nltk  # Thư viện xử lý ngôn ngữ tự nhiên
from nltk.tokenize import word_tokenize  # Để chia văn bản thành các từ
from nltk.corpus import stopwords  # Danh sách các từ không cần thiết ("stop words")
from nltk.stem import WordNetLemmatizer  # Để chuyển các từ về dạng gốc
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import string  # Để làm việc với các phép toán chuỗi cơ bản
import re  # Thư viện xử lý biểu thức chính quy (regular expressions)


# Download stopwords và wordnet nếu chưa có
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')


In [157]:
#import thư viên máy học spark mlib
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.classification import LinearSVC
from pyspark.sql import SparkSession


### Kiểm toán dữ liệu từ tệp CSV dùng pandas


In [158]:
# dùng pandas import dữ liệu 
BankReviews=pd.read_csv("D:\\Niên Luận Cơ Sở_nhóm 2\\BankReviews.csv")

#in ra 5 dòng đầu tiên của dữ liệu
BankReviews.head()


,Date,Stars,Reviews,BankName
0,4/10/2017,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage
1,2/10/2017,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage
2,8/21/2017,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage
3,12/17/2017,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage
4,5/27/2016,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage


In [ ]:
#in ra kích thước của dữ liệu
BankReviews.shape

In [ ]:
#in ra thông tin về dữ liệu
BankReviews.info()

In [ ]:
#kiểm tra xem có giá trị thiếu
BankReviews.isna().sum().sum()

### Xử lí Dữ Liệu


In [ ]:
# loại bỏ cột Data vì nó không dùng
banks=BankReviews.drop(['Date'], axis=1)

In [ ]:
#loại bỏ cột ngân hàng
banks=banks.drop(['BankName'], axis=1)

In [ ]:
banks.head()

In [ ]:
#chuyển chữ hoa sang thường
def text_lowercase(text):
    return text.lower()

banks['Reviews_daxuli']=banks['Reviews'].apply(text_lowercase)

banks.head()

In [ ]:
# Xóa Dấu câu
string.punctuation

In [ ]:

def remove_punct(text):
    text_nonpunc="".join([char for char in text if char not in string.punctuation])
    return text_nonpunc

banks['Reviews_daxuli']=banks['Reviews_daxuli'].apply(remove_punct)

banks.head()

In [ ]:
#Xóa Khoảng Trắng
def remove_whitespaces(text):
    return " ".join(text.split())

banks['Reviews_daxuli']=banks['Reviews_daxuli'].apply(lambda x:remove_whitespaces(x))

banks.head()


#### MÃ HÓA DỮ LIỆU

In [ ]:
#Tách Từ bằng dấu ,
import re

def tokenize(text):
    tokens = re.split(r'\W', text)
    return tokens

banks['Reviews_daxuli'] = banks['Reviews_daxuli'].apply(lambda x: tokenize(x))

banks.head()


In [ ]:
#Xóa stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
nltk.download('vader_lexicon')


In [ ]:
def remove_stopwords(tokenized_list):
    filtered_text=[word for word in tokenized_list if word not in stop]
    return filtered_text

banks['Reviews_daxuli'] = banks['Reviews_daxuli'].apply(lambda x:remove_stopwords(x))

banks.head()

#### Lemmatization

In [ ]:
#Lemmatization kểu nhưng chuyển những từ run, ran, running, runs về từ góc là run

wn=nltk.stem.WordNetLemmatizer()

def lemmatizing(text):
    lemma=[wn.lemmatize(word) for word in text]
    return lemma

banks['Reviews_daxuli'] = banks['Reviews_daxuli'].apply(lemmatizing)

banks.head()

In [ ]:
#Đếm Từ thường gặp
word_count={}

for sentence in banks['Reviews_daxuli']:
    for word in sentence:
        
        if word not in word_count:
            word_count[word]=1
            
        else:
            word_count[word]+=1  
word_count

In [ ]:
#Chuyển đổi word_count thành dataframe

df=pd.DataFrame(word_count.items(), columns=['word','word_count'])
df=df.sort_values('word_count', ascending=False).reset_index(drop=True)
df.head()


### PHÂN TÍCH TÌNH CẢM


In [ ]:
# mức độ tích cực
def sentiment_pos(sentence):
    sent=analyser.polarity_scores(sentence)
    return sent['pos']

# mức độ tiêu cực
def sentiment_neg(sentence):
    sent=analyser.polarity_scores(sentence)
    return sent['neg']

#mức độ trung lập
def sentiment_neu(sentence):
    sent=analyser.polarity_scores(sentence)
    return sent['neu']



In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()
#tính mức tích cực
df['positive']=df['word'].apply(sentiment_pos)

#tính mức tiêu cực
df['negative']=df['word'].apply(sentiment_neg)

#tính mức độ trung lập (TÍNH TRUNG LẬP NÀY KHÔNG CẦN THIẾT CÓ THỂ LOẠI BỎ)
df['neutral'] = df['word'].apply(sentiment_neu)


In [ ]:
df.head()

In [ ]:
# tìm những từ tích cực chính
df1=df.loc[df.positive>0]['word']
df1

In [ ]:
# tìm những từ tiêu cực chính
df2=df.loc[df.negative>0]['word']
df2

In [ ]:
# TÍNH TRUNG LẬP NÀY KHÔNG CẦN THIẾT CÓ THỂ LOẠI BỎ
df3 = df.loc[df['neutral'] > 0]['word']
df3

### PHÂN LOẠI ĐÁNH GIÁ

In [ ]:
banks['positive']=banks['Reviews'].apply(sentiment_pos)
banks['negative']=banks['Reviews'].apply(sentiment_neg)
banks['neutral']=banks['Reviews'].apply(sentiment_neu)

In [ ]:
banks.head()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
comment_words=''
stopwords=set(STOPWORDS)

for x in banks.Reviews: #Lặp qua từng đánh giá
    
    x=str(x) # Chuyển đổi đánh giá thành chuỗi
    
    tokens=x.split() #Tách các từ trong đánh giá
    
    for i in range(len(tokens)): # Lặp qua từng từ 
        tokens[i]=tokens[i].lower() #huyển đổi các từ thành chữ thường
        
    for y in tokens: #Lặp qua từng từ trong danh sách các từ
        comment_words=comment_words+y+' '#Thêm từ vào chuỗi 'comment_words' cùng với một khoảng trắng

In [ ]:
comment_words

In [ ]:
wordcloud=WordCloud(width = 800, height = 800, 
                background_color ='black', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words)

In [ ]:
plt.figure(figsize = (8, 10))
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)

#### XÁC ĐỊNH CÁC CHỦ ĐỀ CHÍNH CỦA VẤN ĐỀ BẰNG THUẬT TOÁN K-MEANS

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

### Sử Dụng Thuật Toán Để Gom Cụm

In [ ]:

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("KMeansExample").getOrCreate()


# Định nghĩa các hàm UDF
sentiment_pos_udf = udf(sentiment_pos, FloatType())
sentiment_neg_udf = udf(sentiment_neg, FloatType())
sentiment_neu_udf = udf(sentiment_neu, FloatType())

# Thêm các cột positive, negative, neutral vào DataFrame banks
banks['positive'] = banks['Reviews'].apply(sentiment_pos)
banks['negative'] = banks['Reviews'].apply(sentiment_neg)
banks['neutral'] = banks['Reviews'].apply(sentiment_neu)

# Chuyển đổi DataFrame pandas thành DataFrame Spark
banks_spark = spark.createDataFrame(banks)

# Lấy các đánh giá tiêu cực để xác định các vấn đề chính
problems = banks_spark.filter(banks_spark['negative'] > 0).select('Reviews')

# Tokenize các đánh giá
tokenizer = Tokenizer(inputCol="Reviews", outputCol="words")
wordsData = tokenizer.transform(problems)

# Tạo vector đặc trưng sử dụng HashingTF
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

# Tính toán IDF cho các vector đặc trưng
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

# Áp dụng thuật toán KMeans để phân cụm
kmeans = KMeans(k=3, seed=42)
model = kmeans.fit(rescaledData)

# Dự đoán phân cụm cho dữ liệu
predictions = model.transform(rescaledData)

# Hiển thị kết quả
predictions.select("Reviews", "prediction").show()

In [ ]:

# Tiếp tục với các bước xử lý kết quả phân cụm
# thống kê số lượng điểm dữ liệu trong mỗi phân cụm
cluster_counts = predictions.groupBy("prediction").count()
cluster_counts.show()


In [ ]:

# Tạo DataFrame từ danh sách các văn bản
df = spark.createDataFrame([(1, "Reviews"), (2, "Reviews_daxuli"), (3, "positive"), (4, "negative"),(5, "neutral")], ["id", "text"])

# Tạo các bước xử lý dữ liệu văn bản trong pipeline
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=20)
idf = IDF(inputCol="raw_features", outputCol="features")

# Tạo pipeline và thực thi các bước xử lý dữ liệu
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, hashingTF, idf])
pipeline_model = pipeline.fit(df)
df_transformed = pipeline_model.transform(df)

# Hiển thị kết quả
df_transformed.show(truncate=False)

#### VECTOR HÓA DỮ LIỆU THÔ: TF-IDF

In [ ]:


# Tạo VectorAssembler để kết hợp các cột features thành một cột vector
assembler = VectorAssembler(inputCols=["features"], outputCol="vector_features")
df_transformed = assembler.transform(df_transformed)

# Lấy dữ liệu vector hóa X từ DataFrame
X = df_transformed.select("vector_features").rdd.map(lambda x: x[0]).collect()

# Số cụm
k = 3

# Tạo mô hình KMeans
kmeans = KMeans(k=k, seed=123)

# Huấn luyện mô hình
model = kmeans.fit(df_transformed)

In [ ]:
df_transformed.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler, Tokenizer, Word2Vec
from pyspark.ml.clustering import KMeans

# Chuyển đổi cột "text" thành vector số
tokenizer = Tokenizer(inputCol="text", outputCol="1text_token")
df_transformed = tokenizer.transform(df_transformed)

# Điều chỉnh giá trị minCount
word2vec = Word2Vec(inputCol="1text_token", outputCol="1text_features", minCount=1)
model = word2vec.fit(df_transformed)
df_transformed = model.transform(df_transformed)

# Chuyển đổi cột "words" thành vector số
word2vec = Word2Vec(inputCol="words", outputCol="1words_features", minCount=1)
model = word2vec.fit(df_transformed)
df_transformed = model.transform(df_transformed)

# Tạo VectorAssembler để kết hợp các cột features thành một cột vector
assembler = VectorAssembler(inputCols=["1text_features", "1words_features"], outputCol="1vector_features")
df_transformed = assembler.transform(df_transformed)

# Số cụm
k = 3

# Tạo mô hình KMeans
kmeans = KMeans(k=k, seed=123)

# Huấn luyện mô hình
model = kmeans.fit(df_transformed)

# Lấy kết quả cluster từ mô hình
predictions = model.transform(df_transformed)

# Tính toán kích thước của các cụm
cluster_sizes = predictions.groupBy("prediction").count().collect()
cluster_sizes = [row["count"] for row in cluster_sizes]
print("Cluster sizes:", cluster_sizes)

# Truy cập vào trung tâm cụm
centroids = model.clusterCenters()

# In kết quả
print("Cluster centers:")
for i, centroid in enumerate(centroids):
    print("Cluster", i, "mean:", centroid.mean())
    print("Cluster", i, "std:", centroid.std())

In [ ]:
# Số cụm
k = 3

# Tạo mô hình KMeans
kmeans = KMeans(k=k, seed=123)

# Huấn luyện mô hình
model = kmeans.fit(df_transformed)

# Truy cập vào trung tâm cụm
centroids = model.clusterCenters()

# In kết quả
for i in range(0, k):
    print('\n\nCluster %d:' % i)
    for ind in centroids[i][:10]:
        print('\t  - %s' % ind)

In [ ]:
predictions.printSchema()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Example").getOrCreate()

# Chuyển đổi DataFrame Pandas 'banks' sang Spark DataFrame
spark_df = spark.createDataFrame(banks)

# Sử dụng Tokenizer để tách từ
tokenizer = Tokenizer(inputCol="Reviews", outputCol="words")
wordsData = tokenizer.transform(spark_df)

# Sử dụng HashingTF để tạo các tính năng từ từ vựng
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2000)
featurizedData = hashingTF.transform(wordsData)

# Sử dụng IDF để tính toán trọng số IDF cho các tính năng
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
transformed_banks = idfModel.transform(featurizedData)

# Huấn luyện mô hình KMeans
kmeans = KMeans(k=3, seed=42)
model = kmeans.fit(transformed_banks)

# Sử dụng mô hình KMeans đã huấn luyện để dự đoán cụm
predictions = model.transform(transformed_banks)

# Thêm cột "cluster" vào DataFrame "predictions" từ cột "prediction"
predictions_with_cluster = predictions.withColumn("cluster", col("prediction"))

# Hiển thị một số dòng đầu tiên của DataFrame "predictions_with_cluster"
predictions_with_cluster.show()

# Gán cụm dự đoán vào một cột mới 'cluster' trong DataFrame
banks_with_clusters = predictions.withColumn("cluster", col("prediction"))

# Hiển thị một số dòng đầu tiên của DataFrame 'banks_with_clusters'
banks_with_clusters.show()

#### KỸ THUẬT HỒI QUY LOGISTIC

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Tạo SparkSession
spark = SparkSession.builder.getOrCreate()

# Tạo DataFrame từ danh sách các từ điển banks
data = spark.createDataFrame(banks, ["label"])

# Lọc dữ liệu và loại bỏ các hàng chứa giá trị null trong cột "label"
data = data.filter(col("label").isNotNull())

# Chuyển đổi kiểu dữ liệu của cột "label" thành số nguyên (integer)
data = data.withColumn("label", col("label").cast("integer"))

# Tạo cột features từ các cột khác trong DataFrame
assembler = VectorAssembler(inputCols=["label"], outputCol="features")
data = assembler.transform(data)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)

# Tạo mô hình Logistic Regression
logistic_regression = LogisticRegression()

# Huấn luyện mô hình trên tập huấn luyện
model = logistic_regression.fit(train_data)

# Dự đoán trên tập huấn luyện và tập kiểm tra
train_predictions = model.transform(train_data)
test_predictions = model.transform(test_data)

# Đánh giá độ chính xác trên tập huấn luyện và tập kiểm tra
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
train_accuracy = evaluator.evaluate(train_predictions)
test_accuracy = evaluator.evaluate(test_predictions)

# In kết quả
print("train_accuracy:", train_accuracy)
print("test_accuracy:", test_accuracy)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, ArrayType, FloatType
from sklearn.metrics import roc_auc_score

# Định nghĩa hàm UDF để chuyển đổi cột dự đoán thành danh sách các xác suất
udf_to_list = udf(lambda x: [float(i) for i in x], ArrayType(FloatType()))
train_predictions = train_predictions.withColumn("probability_list", udf_to_list("probability"))
test_predictions = test_predictions.withColumn("probability_list", udf_to_list("probability"))

# Chuyển đổi DataFrame thành Pandas DataFrame
train_predictions_pd = train_predictions.select("label", "probability_list").toPandas()
test_predictions_pd = test_predictions.select("label", "probability_list").toPandas()

# Xác định số lượng lớp
num_classes = train_predictions_pd["label"].nunique()

# Kiểm tra số lượng lớp
if num_classes < 2:
    roc_auc_scores = [0]  # Trả về giá trị mặc định khi chỉ có một lớp
else:
    # Tính roc_auc_score cho từng lớp
    roc_auc_scores = []
    for class_label in range(num_classes):
        y_true = (train_predictions_pd["label"] == class_label).astype(int)
        y_score = train_predictions_pd["probability_list"].apply(lambda x: x[class_label])

        # Kiểm tra số lượng lớp trong y_true
        if y_true.sum() > 1:
            roc_auc = roc_auc_score(y_true, y_score)
        else:
            roc_auc = 0  # Trả về giá trị mặc định khi chỉ có một lớp
        roc_auc_scores.append(roc_auc)

# In kết quả roc_auc_score cho từng lớp
for class_label, roc_auc in enumerate(roc_auc_scores):
    print(f"Roc Auc Score of class {class_label}: {roc_auc}")

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Chuyển DataFrame của bạn thành một Temporary View
test_predictions.createOrReplaceTempView("test_predictions")

# Sử dụng SQL query để tính toán ma trận nhầm lẫn
confusion_matrix = spark.sql("""
    SELECT label, prediction, count(*) as count
    FROM test_predictions
    GROUP BY label, prediction
    ORDER BY label, prediction
""")

# Hiển thị ma trận nhầm lẫn
confusion_matrix.show()

In [ ]:
import pyspark.sql.functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import matplotlib.pyplot as plt

# Chuyển DataFrame của bạn thành một Temporary View
test_predictions.createOrReplaceTempView("test_predictions")

# Sử dụng SQL query để tính toán ma trận nhầm lẫn
confusion_matrix = spark.sql("""
    SELECT label, prediction, count(*) as count
    FROM test_predictions
    GROUP BY label, prediction
    ORDER BY label, prediction
""")

# Chuyển đổi DataFrame thành Pandas DataFrame
confusion_matrix_pd = confusion_matrix.toPandas()

# Tạo ma trận nhầm lẫn bằng cách sử dụng cột "count" trong Pandas DataFrame
cm = confusion_matrix_pd.pivot(index='label', columns='prediction', values='count')

# Hiển thị ma trận nhầm lẫn bằng Heatmap
plt.imshow(cm, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.xticks([0, 1], ["1", "5"])
plt.yticks([0, 1], ["1", "5"])
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion matrix')
plt.show()

In [ ]:
from pyspark.sql.functions import lit

# Thêm cột predicted_stars vào DataFrame banks_with_clusters
final_df = banks_with_clusters.withColumn("predicted_stars", lit(banks_with_clusters["Stars"]))

final_df = final_df.drop("words", "rawFeatures", "features", "prediction","cluster")

# Hiển thị bảng cuối cùng final_df
final_df.show()